In [5]:
#default_exp onehot

In [6]:
#export
from collections import OrderedDict
import pandas as pd
import numpy as np
from food.tools import from_pickle, to_pickle

In [7]:
#export
class OneHot:
    def __init__(self, bins):
        self.bins = bins
        self.n_bin = OrderedDict(((k,len(v)-1) for k,v in bins.items()))
           
    @classmethod
    def from_df(cls, n_bin:OrderedDict, df:pd.DataFrame, apply:bool=False):
        bins = OrderedDict()
        for c in n_bin.keys(): 
            c_binned, bins[c] =  pd.qcut(df[c], n_bin[c], retbins=True, duplicates='drop')   
            bins[c][0], bins[c][-1] = -np.Inf, np.Inf
            if apply: df[c] = c_binned
        return cls(bins)
    
    @classmethod
    def from_file(cls, fname:str):
        return cls(from_pickle(fname))
    
    def n_dim(self):
        return sum(list(self.n_bin.values()))
    
    def to_file(self, fname:str):
        to_pickle(self.bins, fname)

    def apply_bins(self, df:pd.DataFrame):
        for c,abin in self.bins.items(): 
            df[c] = pd.cut(df[c], bins=abin)

    def apply(self, df:pd.DataFrame, target='clip'): #this works for vectors in single column
        df_temp = pd.DataFrame(df[target].tolist())
        for c, abin in self.bins.items(): 
            binned_column = pd.cut(df[c], bins=abin)
            one_hot_columns = pd.get_dummies(binned_column, prefix=f'target_{c}')
            df_temp = df_temp.join(one_hot_columns)
        
        v_onehot = df_temp.apply(lambda row:row.values,axis=1)
        df[target] = v_onehot

    
        
    def apply_multiple_columns(self, df:pd.DataFrame, target='clip'): #this works for vectors destributed between multiple columns
        df_temp = df
        for c, abin in self.bins.items(): 
            binned_column = pd.cut(df[c], bins=abin)
            one_hot_columns = pd.get_dummies(binned_column, prefix=f'{target}_{c}')
            df_temp = df_temp.join(one_hot_columns)
        
        v_onehot = df_temp[[c for c in df_temp.columns if target in c]].apply(lambda row:row.values,axis=1)
        df.drop(columns = [c for c in df.columns if target in c], inplace=True)
        df[target] = v_onehot
        
    def build_milvus_query(self, parameters):
        bins, n_bin = self.bins, self.n_bin
        query_expr = ""
        sep = ""

        mask_query = OrderedDict()
        for c,dim in n_bin.items():
            mask_query[c] = np.ones(dim)  

        def min_one_hot(p_name, p_val):
            if p_name not in bins: return
            mul = p_val <= bins[p_name]
            mul = mul[1:]
            mask_query[p_name] *= mul

        def max_one_hot(p_name, p_val):
            if p_name not in bins: return
            mul = bins[p_name] < p_val
            mul = mul[:-1]
            mask_query[p_name] *= mul 

        def eq_one_hot(p_name, p_val):
            if p_name not in bins: return
            a = p_val <= bins[p_name] 
            b = bins[p_name] < p_val
            mul = a[1:]*b[:-1]
            mask_query[p_name] *= mul 

        for p_name, p_val in parameters.items():
            if p_val is None: continue
            if p_name.startswith('min_'):
                p_name = p_name.removeprefix('min_')
                min_one_hot(p_name, p_val)
                query_expr += sep + f"{p_name} >= {p_val}"
            elif p_name.startswith('max_'):
                p_name = p_name.removeprefix('max_')
                max_one_hot(p_name, p_val)
                query_expr += sep + f"{p_name} <= {p_val}"
            else:
                eq_one_hot(p_name, p_val)
                query_expr += sep + f"{p_name} == {p_val}"
            sep = " && "

        for c,dim in n_bin.items():
            if all(mask_query[c]):
                mask_query[c] = np.zeros(dim)  
            
        return query_expr, np.hstack(list(mask_query.values()))

In [8]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_search copy.ipynb.
Converted 01_search.ipynb.
Converted 02_onehot.ipynb.
Converted colr_segmentation.ipynb.
Converted food_clustering.ipynb.
Converted inference.ipynb.
Converted search_segmented.ipynb.
Converted search_segmented_stable.ipynb.
Converted stats.ipynb.
Converted update_qdrant.ipynb.
Converted usda_parsing.ipynb.
Converted weird_results_of_image2image.ipynb.
